In [156]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (126 kB/s)
Reading package lists... Done


In [136]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-10-12 01:02:07--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  3.88MB/s    in 0.2s    

2020-10-12 01:02:08 (3.88 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [137]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [138]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [154]:
votes_over_20_df= df.filter("total_votes>=20")
votes_over_20_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   17728960|R14W2VCHHK5V7W|B00ZI3R5VG|     213303319|Stuhrling Origina...|         Watches|          5|           19|         20|   N|                Y|this is a good al...|As a former Rolex...| 2015-08-31|
|         US|   20240757|R1S3T57O3OYT5S|B003DZ7VOW|     336197576|G-Shock GA100A-7A...|         Watches|          5|    

In [172]:
from pyspark .sql.functions import *

percenthelpful_votes= votes_over_20_df.select(((col("helpful_votes"))/(col("total_votes")))*100)
percent_helpful_votes_df= votes_over_20_df.withColumn('percenthelpful_votes',(col('helpful_votes')/(col('total_votes')) *100))
percent_helpful_votes_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|percenthelpful_votes|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+
|         US|   17728960|R14W2VCHHK5V7W|B00ZI3R5VG|     213303319|Stuhrling Origina...|         Watches|          5|           19|         20|   N|                Y|this is a good al...|As a former Rolex...| 2015-08-31|                95.0|
|         US|   20240757|R1S3T57O3OY

In [173]:
percent_votes_df = percent_helpful_votes_df.filter("percenthelpful_votes >=50").show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|percenthelpful_votes|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+
|         US|   17728960|R14W2VCHHK5V7W|B00ZI3R5VG|     213303319|Stuhrling Origina...|         Watches|          5|           19|         20|   N|                Y|this is a good al...|As a former Rolex...| 2015-08-31|                95.0|
|         US|   20240757|R1S3T57O3OY

In [174]:
percent_helpful_votes_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|percenthelpful_votes|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+
|         US|   17728960|R14W2VCHHK5V7W|B00ZI3R5VG|     213303319|Stuhrling Origina...|         Watches|          5|           19|         20|   N|                Y|this is a good al...|As a former Rolex...| 2015-08-31|                95.0|
|         US|   20240757|R1S3T57O3OY

In [177]:
vine_paid_df = percent_helpful_votes_df.filter("vine =='Y'")
vine_paid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|percenthelpful_votes|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+
|         US|   53096363|R1B7M0OP3UNP6O|B00V0G9OGE|     297682929|Armitron Sport Wo...|         Watches|          5|           49|         52|   Y|                N|Lots of features ...|My wife has been ...| 2015-08-27|   94.23076923076923|
|         US|   51394788|R2UUV4UGGYM

In [178]:
vine_notpaid_df= percent_helpful_votes_df.filter("vine == 'N'").show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|percenthelpful_votes|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+
|         US|   17728960|R14W2VCHHK5V7W|B00ZI3R5VG|     213303319|Stuhrling Origina...|         Watches|          5|           19|         20|   N|                Y|this is a good al...|As a former Rolex...| 2015-08-31|                95.0|
|         US|   20240757|R1S3T57O3OY

In [179]:
df.select("review_id").count()

960872

In [180]:
df.filter("star_rating = 5").count()


571627

In [197]:
paid_rating = vine_paid_df.filter("star_rating = 5")
paid_rating.count()



15

In [205]:
paid_review_id = vine_paid_df.select("review_id")
paid_review_id.count()

47

In [220]:
percent_paid_ratings = (paid_rating/paid_review_id)*100
percent_paid_ratings


TypeError: ignored

In [221]:
notpaid_rating = vine_notpaid_df.filter("star_rating=5")
notpaid_rating.count()

AttributeError: ignored

In [219]:
notpaid_review_id = vine_notpaid_df.select("review_id")
notpaid_review_id.count()


AttributeError: ignored